<a href="https://colab.research.google.com/github/zweiner/Awesome/blob/main/Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from typing import Dict
def load_dataset_from_hdf(filepath: str) -> Dict:
    dataset = {}
    with pd.HDFStore(filepath, mode='r') as store:
        keys = store.keys()
        for key in keys:
            # Parse the key structure: /p20/gsr/file01 -> person=20, type=gsr, file=1
            parts = key.split('/')
            person_num = int(parts[1].replace('p', ''))
            data_type = parts[2]
            if person_num not in dataset:
                dataset[person_num] = {'gsr': [], 'ppg': []}
            df = store.get(key)
            dataset[person_num][data_type].append(df)
        for person_data in dataset.values():
            for data_type in ['gsr', 'ppg']:
                person_data[data_type].sort(key=lambda x: len(person_data[data_type]))
    return dataset

In [ ]:
dataset = load_dataset_from_hdf(filepath='../full_dataset.h5')
dataset[20]['ppg'][0]
# person 20 ppg and they are angry

In [ ]:
dataset[22]

In [ ]:
import matplotlib.pyplot as plt

def plot_simple_waveform(data):
    """
    Simple function to plot waveform data

    Parameters:
    data : array-like
        The data to plot
    """
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(data, linewidth=1)
    ax.grid(True, alpha=0.3)
    ax.set_title('Waveform')
    plt.tight_layout()
    plt.show()

    return fig, ax

# Example usage
# Replace `dataset[24]['ppg'][1].iloc[:-1]` with your actual data
plot_simple_waveform(dataset[24]['gsr'][1].iloc[:-1])


In [ ]:
# Plotting all the formants all at once

import pandas as pd
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
for column in formant_df.columns[1:]:
    plt.plot(formant_df['Time'], formant_df[column], label=column)

plt.xlabel('Time (s)')
plt.ylabel('Formant Frequency (Hz)')
plt.title('Formant Features over Time')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))

plt.plot(formant_df['Time'], formant_df['pf'], label=column)

plt.xlabel('Time (s)')
plt.ylabel('Formant Frequency (Hz)')
plt.title('Formant Features over Time')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


# GSR data from the provided code
# Replace `dataset[24]['gsr'][1].iloc[:-1]` with your actual GSR data
gsr_data = dataset[24]['gsr'][1].iloc[:-1].values.flatten()

# Creating time array for GSR data
time_gsr = np.linspace(formant_df['Time'].min(), formant_df['Time'].max(), len(gsr_data))

# Interpolate GSR data to match the time axis of formant_df
gsr_interpolated = np.interp(formant_df['Time'], time_gsr, gsr_data)

# Ensure formant_df['pf'] is a 1D array
pf_data = formant_df['F1_Frequency'].values.flatten()

# Compute cross-correlation
cross_correlation = np.correlate(gsr_interpolated - np.mean(gsr_interpolated), pf_data, mode='full')

# Calculate time lags
lags = np.arange(-len(pf_data) + 1, len(gsr_interpolated))

# Find the time lag with the maximum cross-correlation value
peak_index = np.argmax(cross_correlation)
peak_time_lag = lags[peak_index] * (formant_df['Time'][1] - formant_df['Time'][0])

# Plotting
plt.figure(figsize=(10, 6))

# Plot original pf formant feature
plt.subplot(3, 1, 1)
plt.plot(formant_df['Time'], formant_df['F1_Frequency'], label='pf')
plt.xlabel('Time (s)')
plt.ylabel('Formant Frequency (Hz)')
plt.title('Formant Feature over Time')
plt.legend()
plt.grid(True)

# Plot GSR data
plt.subplot(3, 1, 2)
plt.plot(np.linspace(formant_df['Time'].min(), formant_df['Time'].max(), len(gsr_data)), gsr_data, label='gsr', color='orange')
plt.xlabel('Time (s)')
plt.ylabel('GSR')
plt.title('GSR Data over Time')
plt.legend()
plt.grid(True)

# Plot Cross-correlation
plt.subplot(3, 1, 3)
plt.plot(lags * (formant_df['Time'][1] - formant_df['Time'][0]), cross_correlation, label='Cross-correlation', color='green')
plt.axvline(peak_time_lag, color='red', linestyle='--', label=f'Peak Lag: {peak_time_lag:.2f}s')
plt.xlabel('Time Lag (s)')
plt.ylabel('Cross-correlation')
plt.title('Cross-correlation of Formant Feature and GSR')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Peak time lag value
print(f'The time lag that gives the peak value is {peak_time_lag:.2f} seconds.')







In [ ]:
file_path = "Audio-24-01-03.wav"

In [ ]:

# formant data frame that works with
import numpy as np
import pandas as pd
import parselmouth
import statistics
from parselmouth.praat import call
from scipy.stats.mstats import zscore
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

f0min = 75
# file_path = "the_north_wind_and_the_sun.wav"

def measurePitch(sound, start_time, end_time, f0min, f0max, unit):
    # Extract the window of audio
    window = sound.extract_part(start_time, end_time)

    duration = end_time - start_time

    # Calculate the required minimum pitch based on the window size
    required_min_pitch = max(f0min, 1 / duration)

    # Ensure the window size is practical
    if duration < 1 / f0min * 1:  # Adjust the multiplier as needed
        raise ValueError(f"Window size too small for minimum pitch {f0min} Hz. Increase the window size.")

    pitch = call(window, "To Pitch", 0.0, required_min_pitch, f0max)
    meanF0 = call(pitch, "Get mean", 0, 0, unit)
    stdevF0 = call(pitch, "Get standard deviation", 0, 0, unit)
    harmonicity = call(window, "To Harmonicity (cc)", 0.01, required_min_pitch, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    pointProcess = call(window, "To PointProcess (periodic, cc)", required_min_pitch, f0max)

    # Measure jitter
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)

    # Measure shimmer
    localShimmer = call([window, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([window, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([window, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([window, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer = call([window, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    ddaShimmer = call([window, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)

    return (duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter,
            ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer)

def measure_formants_to_df(file_path, time_step=0.03, max_formants=5, formant_ceiling=5500, window_length=0.03, pre_emphasis_from=50):
    # Load the sound from the file path
    sound = parselmouth.Sound(file_path)

    # Resample the sound to twice the formant ceiling
    sound = call(sound, "Resample", 2 * formant_ceiling, 50)

    # Apply pre-emphasis
    call(sound, "Pre-emphasize (in-place)", pre_emphasis_from)

    # Create a Formant object
    formants = call(sound, "To Formant (burg)", time_step, max_formants, formant_ceiling, window_length, pre_emphasis_from)

    # Extract formant frequencies and bandwidths
    num_frames = call(formants, "Get number of frames")
    data = []

    for frame in range(1, num_frames + 1):
        frame_time = (frame - 1) * time_step
        frame_data = {'Time': frame_time}
        for formant_number in range(1, max_formants + 1):
            frequency = call(formants, "Get value at time", formant_number, frame_time, 'Hertz', 'Linear')
            bandwidth = call(formants, "Get bandwidth at time", formant_number, frame_time, 'Hertz', 'Linear')
            frame_data[f'F{formant_number}_Frequency'] = frequency
            frame_data[f'F{formant_number}_Bandwidth'] = bandwidth
        data.append(frame_data)

    # Create a DataFrame
    df = pd.DataFrame(data)

    return df

def measureFormants(sound, wave_file, f0min, f0max):
    sound = parselmouth.Sound(sound)  # read the sound
    pitch = call(sound, "To Pitch (cc)", 0, f0min, 15, 'no', 0.03, 0.45, 0.01, 0.35, 0.14, f0max)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)

    formants = call(sound, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)
    numPoints = call(pointProcess, "Get number of points")

    f1_list = []
    f2_list = []
    f3_list = []
    f4_list = []

    # Measure formants only at glottal pulses
    for point in range(0, numPoints):
        point += 1
        t = call(pointProcess, "Get time from index", point)
        f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
        f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
        f3 = call(formants, "Get value at time", 3, t, 'Hertz', 'Linear')
        f4 = call(formants, "Get value at time", 4, t, 'Hertz', 'Linear')
        f1_list.append(f1)
        f2_list.append(f2)
        f3_list.append(f3)
        f4_list.append(f4)

    f1_list = [f1 for f1 in f1_list if str(f1) != 'nan']
    f2_list = [f2 for f2 in f2_list if str(f2) != 'nan']
    f3_list = [f3 for f3 in f3_list if str(f3) != 'nan']
    f4_list = [f4 for f4 in f4_list if str(f4) != 'nan']

    # calculate mean formants across pulses
    f1_mean = statistics.mean(f1_list)
    f2_mean = statistics.mean(f2_list)
    f3_mean = statistics.mean(f3_list)
    f4_mean = statistics.mean(f4_list)

    # calculate median formants across pulses, this is what is used in all subsequent calculations
    f1_median = statistics.median(f1_list)
    f2_median = statistics.median(f2_list)
    f3_median = statistics.median(f3_list)
    f4_median = statistics.median(f4_list)

    return f1_mean, f2_mean, f3_mean, f4_mean, f1_median, f2_median, f3_median, f4_median

#file_path = "the_north_wind_and_the_sun.wav"
formant_df = measure_formants_to_df(file_path)

# Load the sound from the file path
sound = parselmouth.Sound(file_path)

# Calculate additional features using measureFormants
f1_mean, f2_mean, f3_mean, f4_mean, f1_median, f2_median, f3_median, f4_median = measureFormants(sound, file_path, f0min, 500)


#file_path = "the_north_wind_and_the_sun.wav"
formant_df = measure_formants_to_df(file_path)

# Load the sound from the file path
sound = parselmouth.Sound(file_path)

# Calculate additional features using measureFormants
f1_mean, f2_mean, f3_mean, f4_mean, f1_median, f2_median, f3_median, f4_median = measureFormants(sound, file_path, f0min, 500)


formant_df['F1_z'] = (formant_df['F1_Frequency'] - formant_df['F1_Frequency'].mean()) / formant_df['F1_Frequency'].std()
formant_df['F2_z'] = (formant_df['F2_Frequency'] - formant_df['F2_Frequency'].mean()) / formant_df['F2_Frequency'].std()
formant_df['F3_z'] = (formant_df['F3_Frequency'] - formant_df['F3_Frequency'].mean()) / formant_df['F3_Frequency'].std()
formant_df['F4_z'] = (formant_df['F4_Frequency'] - formant_df['F4_Frequency'].mean()) / formant_df['F4_Frequency'].std()

# Calculate pf using the z-scores of F1, F2, F3, and F4
formant_df['pf'] = (formant_df['F1_z'] + formant_df['F2_z'] + formant_df['F3_z'] + formant_df['F4_z']) / 4

# Print the median values
print(f"Median of F1: {f1_median}")
print(f"Median of F2: {f2_median}")
print(f"Median of F3: {f3_median}")
print(f"Median of F4: {f4_median}")

# Display the dataframe without median values
print(formant_df)

In [ ]:
formant_df

In [ ]:
#shows you that calling parsel mouth is what takes up all the time
import cProfile

def main():
    file_path = "the_north_wind_and_the_sun.wav"
    formant_df = measure_formants_to_df(file_path)
    sound = parselmouth.Sound(file_path)
    f1_mean, f2_mean, f3_mean, f4_mean, f1_median, f2_median, f3_median, f4_median = measureFormants(sound, file_path, f0min, 500)
    print(formant_df)

cProfile.run('main()', sort='cumtime')

In [ ]:
# try to use GPU acceleration on Formants
import numpy as np
import pandas as pd
import parselmouth
import statistics
from parselmouth.praat import call
import cupy as cp  # GPU-accelerated NumPy-like library

# Constants
f0min = 75
f0max = 500

def measurePitch(sound, start_time, end_time, f0min, f0max, unit):
    # Extract the window of audio
    window = sound.extract_part(start_time, end_time)
    duration = end_time - start_time

    # Calculate the required minimum pitch based on the window size
    required_min_pitch = max(f0min, 1 / duration)

    # Ensure the window size is practical
    if duration < 1 / f0min * 1:  # Adjust the multiplier as needed
        raise ValueError(f"Window size too small for minimum pitch {f0min} Hz. Increase the window size.")

    pitch = call(window, "To Pitch", 0.0, required_min_pitch, f0max)
    meanF0 = call(pitch, "Get mean", 0, 0, unit)
    stdevF0 = call(pitch, "Get standard deviation", 0, 0, unit)
    harmonicity = call(window, "To Harmonicity (cc)", 0.01, required_min_pitch, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    pointProcess = call(window, "To PointProcess (periodic, cc)", required_min_pitch, f0max)

    # Measure jitter and shimmer
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)

    localShimmer = call([window, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([window, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([window, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([window, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer = call([window, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    ddaShimmer = call([window, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)

    return (duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter,
            ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer)

def measure_formants_to_df(file_path, time_step=0.03, max_formants=5, formant_ceiling=5500, window_length=0.03, pre_emphasis_from=50):
    """
    Measure formants and return a DataFrame.
    """
    # Load the sound from the file path
    sound = parselmouth.Sound(file_path)

    # Resample the sound to twice the formant ceiling
    sound = call(sound, "Resample", 2 * formant_ceiling, 50)

    # Apply pre-emphasis
    call(sound, "Pre-emphasize (in-place)", pre_emphasis_from)

    # Create a Formant object
    formants = call(sound, "To Formant (burg)", time_step, max_formants, formant_ceiling, window_length, pre_emphasis_from)

    # Extract formant frequencies and bandwidths
    num_frames = call(formants, "Get number of frames")
    data = []

    for frame in range(1, num_frames + 1):
        frame_time = (frame - 1) * time_step
        frame_data = {'Time': frame_time}
        for formant_number in range(1, max_formants + 1):
            frequency = call(formants, "Get value at time", formant_number, frame_time, 'Hertz', 'Linear')
            bandwidth = call(formants, "Get bandwidth at time", formant_number, frame_time, 'Hertz', 'Linear')
            frame_data[f'F{formant_number}_Frequency'] = frequency
            frame_data[f'F{formant_number}_Bandwidth'] = bandwidth
        data.append(frame_data)

    # Create a DataFrame
    df = pd.DataFrame(data)

    # Offload z-score calculations to the GPU using CuPy
    for formant_number in range(1, max_formants + 1):
        freq_col = f'F{formant_number}_Frequency'
        freq_gpu = cp.asarray(df[freq_col].values)
        z_scores_gpu = (freq_gpu - cp.mean(freq_gpu)) / cp.std(freq_gpu)
        df[f'{freq_col}_z'] = cp.asnumpy(z_scores_gpu)

    # Calculate pf using the z-scores of F1, F2, F3, and F4
    df['pf'] = (df['F1_Frequency_z'] + df['F2_Frequency_z'] + df['F3_Frequency_z'] + df['F4_Frequency_z']) / 4

    return df

# Example usage
file_path = "the_north_wind_and_the_sun.wav"
formant_df = measure_formants_to_df(file_path)

# Display the dataframe
print(formant_df)

In [ ]:
import numpy as np
import pandas as pd
import parselmouth
import statistics
from parselmouth.praat import call
from multiprocessing import Pool, RawArray
import ctypes

# Constants
f0min = 75
f0max = 500

# Shared memory for the sound data
shared_sound_data = None

def init_worker(sound_data, sample_rate):
    """
    Initialize the worker with shared sound data.
    """
    global shared_sound_data
    shared_sound_data = parselmouth.Sound(sound_data, sample_rate)

def process_frame(frame, time_step, max_formants, formant_ceiling, window_length, pre_emphasis_from):
    """
    Process a single frame using shared sound data.
    """
    global shared_sound_data
    sound = shared_sound_data

    frame_time = (frame - 1) * time_step
    frame_data = {'Time': frame_time}

    # Create a Formant object
    formants = call(sound, "To Formant (burg)", time_step, max_formants, formant_ceiling, window_length, pre_emphasis_from)

    for formant_number in range(1, max_formants + 1):
        frequency = call(formants, "Get value at time", formant_number, frame_time, 'Hertz', 'Linear')
        bandwidth = call(formants, "Get bandwidth at time", formant_number, frame_time, 'Hertz', 'Linear')
        frame_data[f'F{formant_number}_Frequency'] = frequency
        frame_data[f'F{formant_number}_Bandwidth'] = bandwidth

    return frame_data

def measure_formants_to_df(file_path, time_step=0.03, max_formants=5, formant_ceiling=5500, window_length=0.03, pre_emphasis_from=50):
    """
    Measure formants and return a DataFrame.
    """
    # Load the sound file
    sound = parselmouth.Sound(file_path)
    total_duration = sound.get_total_duration()
    sample_rate = sound.get_sampling_frequency()

    # Convert sound data to a NumPy array for shared memory
    sound_array = sound.values.T  # Transpose to match Praat's format
    shared_array = RawArray(ctypes.c_double, sound_array.size)
    np_array = np.frombuffer(shared_array, dtype=np.float64).reshape(sound_array.shape)
    np.copyto(np_array, sound_array)

    # Calculate the number of frames
    num_frames = int(total_duration / time_step)

    # Initialize the worker pool with shared sound data
    with Pool(initializer=init_worker, initargs=(shared_array, sample_rate)) as pool:
        # Process frames in parallel
        data = pool.starmap(
            process_frame,
            [(frame, time_step, max_formants, formant_ceiling, window_length, pre_emphasis_from) for frame in range(1, num_frames + 1)]
        )

    # Create a DataFrame
    df = pd.DataFrame(data)
    return df

# Example usage
file_path = "the_north_wind_and_the_sun.wav"
formant_df = measure_formants_to_df(file_path)

# Display the dataframe
print(formant_df)

In [ ]:
import matplotlib.pyplot as plt

# Define a function to plot the formant values
def plot_formants(formant_df):
    plt.figure(figsize=(14, 8))

    # Plot the formant frequencies
    plt.subplot(2, 1, 1)
    plt.plot(formant_df['Time'], formant_df['F1_Frequency'], label='F1 Frequency', color='r')
    plt.plot(formant_df['Time'], formant_df['F2_Frequency'], label='F2 Frequency', color='g')
    plt.plot(formant_df['Time'], formant_df['F3_Frequency'], label='F3 Frequency', color='b')
    plt.plot(formant_df['Time'], formant_df['F4_Frequency'], label='F4 Frequency', color='m')
    plt.title('Formant Frequencies')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.legend()

    # Plot the z-scores of the formant frequencies
    plt.subplot(2, 1, 2)
    plt.plot(formant_df['Time'], formant_df['F1_z'], label='F1 z-score', color='r')
    plt.plot(formant_df['Time'], formant_df['F2_z'], label='F2 z-score', color='g')
    plt.plot(formant_df['Time'], formant_df['F3_z'], label='F3 z-score', color='b')
    plt.plot(formant_df['Time'], formant_df['F4_z'], label='F4 z-score', color='m')
    plt.title('Z-scores of Formant Frequencies')
    plt.xlabel('Time (s)')
    plt.ylabel('Z-score')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Call the plotting function
plot_formants(formant_df)


In [ ]:
#HNR
import parselmouth
from parselmouth.praat import call
import pandas as pd

def measure_hnr_to_df(file_path, window_length=0.03, time_step=0.03):
    # Load the sound from the file path
    sound = parselmouth.Sound(file_path)

    # Get the total duration of the sound
    total_duration = sound.get_total_duration()

    # Initialize an empty list to store HNR values and corresponding times
    data = []

    # Iterate over the sound in windows of specified length
    for start_time in range(0, int(total_duration / time_step)):
        start_time = start_time * time_step
        end_time = start_time + window_length

        # Extract the window of audio
        window = sound.extract_part(from_time=start_time, to_time=end_time)

        # Calculate Harmonic-to-Noise Ratio (HNR)
        harmonicity = call(window, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
        hnr = call(harmonicity, "Get mean", 0, 0)

        # Append the HNR value and corresponding time to the list
        data.append({'Time': start_time, 'HNR': hnr})

    # Create a DataFrame from the list
    df = pd.DataFrame(data)

    return df

# Example usage
file_path = "Audio-24-01-03.wav"
hnr_df = measure_hnr_to_df(file_path)

hnr_df

In [ ]:
import matplotlib.pyplot as plt

# Define a function to plot the HNR values
def plot_hnr(hnr_df):
    plt.figure(figsize=(10, 6))
    plt.plot(hnr_df['Time'], hnr_df['HNR'], label='HNR', color='blue')
    plt.title('Harmonic-to-Noise Ratio (HNR) Over Time')
    plt.xlabel('Time (s)')
    plt.ylabel('HNR (dB)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Example usage
plot_hnr(hnr_df)


In [ ]:
#Fundamental frequency calculations--> Notice that Jitter is using F0 (with the same pitch parameters) to get its calculation
import pandas as pd
import parselmouth
from parselmouth.praat import call

def measure_f0_to_df(file_path, time_step=0, f0min=75, f0max=600):
    # Load the sound from the file path
    sound = parselmouth.Sound(file_path)

    sound = parselmouth.Sound(sound)  # read the sound
    pitch = call(sound, "To Pitch (cc)", 0, f0min, 15, 'no', 0.03, 0.45, 0.01, 0.35, 0.14, f0max)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)

    # Extract pitch values
    num_frames = call(pitch, "Get number of frames")
    print(f"Number of frames: {num_frames}")
    data = []

    for frame in range(1, num_frames + 1):
        frame_time = call(pitch, "Get time from frame number", frame)
        f0_value = call(pitch, "Get value at time", frame_time, 'Hertz', 'Linear')
        data.append({'Time': frame_time, 'F0': f0_value})

    # Create a DataFrame
    df = pd.DataFrame(data)

    return df

# Example usage
file_path = "Audio-24-01-03.wav"
# file_path = "440Hz_44100Hz_16bit_05sec.wav"
f0_df = measure_f0_to_df(file_path)
row_number = 100  # Replace with the row number you're interested in

# Check if the row number is within the DataFrame's range
if 0 <= row_number < len(f0_df):
    f0_value = f0_df.iloc[row_number]['F0']
    print(f"F0 value at row {row_number}: {f0_value}")
else:
    print(f"Row number {row_number} is out of range. Please choose a row number between 0 and {len(f0_df) - 1}.")

f0_df

#nan values are due to the nature of the signal (uncomment the other file path to verify this); num of frames and time window are now aligned
# this is verfied by the number of rows being the same as the number of frames
# you can get rid of the nan values by adjusting f0max and f0min --> note the cell below

In [ ]:
import matplotlib.pyplot as plt

# Define a function to plot the F0 values
def plot_f0(f0_df):
    plt.figure(figsize=(10, 6))
    plt.plot(f0_df['Time'], f0_df['F0'], label='F0', color='blue')
    plt.title('Fundamental Frequency (F0) Over Time')
    plt.xlabel('Time (s)')
    plt.ylabel('F0 (Hz)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Example usage
plot_f0(f0_df)


In [ ]:
import matplotlib.pyplot as plt

def plot_simple_waveform(data):
    """
    Simple function to plot waveform data

    Parameters:
    data : array-like
        The data to plot
    """
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(data, linewidth=1)
    ax.grid(True, alpha=0.3)
    ax.set_title('Waveform')
    plt.tight_layout()
    plt.show()

    return fig, ax

# Example usage
# Replace `dataset[24]['ppg'][1].iloc[:-1]` with your actual data
plot_simple_waveform(dataset[24]['gsr'][1].iloc[:-1])


In [ ]:
#updated code with GPU acceleration

import numpy as np
import pandas as pd
import parselmouth
import statistics
from parselmouth.praat import call
import cupy as cp  # GPU-accelerated NumPy-like library

# Constants
f0min = 75
f0max = 500

def measurePitch(sound, start_time, end_time, f0min, f0max, unit):
    # Extract the window of audio
    window = sound.extract_part(start_time, end_time)
    duration = end_time - start_time

    # Calculate the required minimum pitch based on the window size
    required_min_pitch = max(f0min, 1 / duration)

    # Ensure the window size is practical
    if duration < 1 / f0min * 1:  # Adjust the multiplier as needed
        raise ValueError(f"Window size too small for minimum pitch {f0min} Hz. Increase the window size.")

    pitch = call(window, "To Pitch", 0.0, required_min_pitch, f0max)
    meanF0 = call(pitch, "Get mean", 0, 0, unit)
    stdevF0 = call(pitch, "Get standard deviation", 0, 0, unit)
    harmonicity = call(window, "To Harmonicity (cc)", 0.01, required_min_pitch, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    pointProcess = call(window, "To PointProcess (periodic, cc)", required_min_pitch, f0max)

    # Measure jitter and shimmer
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)

    localShimmer = call([window, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([window, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([window, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([window, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer = call([window, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    ddaShimmer = call([window, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)

    return (duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter,
            ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer)

def measure_formants_to_df(file_path, time_step=0.03, max_formants=5, formant_ceiling=5500, window_length=0.03, pre_emphasis_from=50):
    """
    Measure formants and return a DataFrame.
    """
    # Load the sound from the file path
    sound = parselmouth.Sound(file_path)

    # Resample the sound to twice the formant ceiling
    sound = call(sound, "Resample", 2 * formant_ceiling, 50)

    # Apply pre-emphasis
    call(sound, "Pre-emphasize (in-place)", pre_emphasis_from)

    # Create a Formant object
    formants = call(sound, "To Formant (burg)", time_step, max_formants, formant_ceiling, window_length, pre_emphasis_from)

    # Extract formant frequencies and bandwidths
    num_frames = call(formants, "Get number of frames")
    data = []

    for frame in range(1, num_frames + 1):
        frame_time = (frame - 1) * time_step
        frame_data = {'Time': frame_time}
        for formant_number in range(1, max_formants + 1):
            frequency = call(formants, "Get value at time", formant_number, frame_time, 'Hertz', 'Linear')
            bandwidth = call(formants, "Get bandwidth at time", formant_number, frame_time, 'Hertz', 'Linear')
            frame_data[f'F{formant_number}_Frequency'] = frequency
            frame_data[f'F{formant_number}_Bandwidth'] = bandwidth
        data.append(frame_data)

    # Create a DataFrame
    df = pd.DataFrame(data)

    # Offload z-score calculations to the GPU using CuPy
    for formant_number in range(1, max_formants + 1):
        freq_col = f'F{formant_number}_Frequency'
        freq_gpu = cp.asarray(df[freq_col].values)
        z_scores_gpu = (freq_gpu - cp.mean(freq_gpu)) / cp.std(freq_gpu)
        df[f'{freq_col}_z'] = cp.asnumpy(z_scores_gpu)

    # Calculate pf using the z-scores of F1, F2, F3, and F4
    df['pf'] = (df['F1_Frequency_z'] + df['F2_Frequency_z'] + df['F3_Frequency_z'] + df['F4_Frequency_z']) / 4

    return df

# Example usage
file_path = "the_north_wind_and_the_sun.wav"
formant_df = measure_formants_to_df(file_path)

# Display the dataframe
print(formant_df)